steps of a genetic algorithm:

initialization
evaluation
elitism
parent selection
reproduction
mutation

candlestick based genalg params:
modify these based off of largely expanded featurespace

popsize - 200
num gens - 9
elitism copys - 1
mutation prob - 5%
fresh pattern prob - 2%
min number of patterns - 2.5%

In [ ]:
import _9_utility as util

dataset = util.load_large_csv('spx_features_small.csv')


In [ ]:
fssr = util.fn_all_subsets(True,0,True)
fssd = util.fnsubset_to_indexdictlist(dataset.columns,fssr)
fss = []
for d in fssd:
    fss.append(list(d.keys()))
print(fss)

In [ ]:
dataset = dataset.values
arr_low = dataset[:,1]
arr_close = dataset[:,2]

In [ ]:
#example for initiation and evaluation steps

from importlib import reload
import _0_gene as _0
reload(_0)
import _1_initialization as _1
reload(_1)
import _2_evaluation as _2
reload(_2)
import _3_elitism as _3
import _4_parentselection as _4
reload(_4)
import _5_reproduction as _5
reload(_5)
import _6_mutation as _6
import _9_utility as util


#dataset = util.load_large_csv('spx_features.csv')
#dataset, fss = util.drop_all_targets()


lag_allowance = 30
generations = 10
hold_for = 10


population = _1.generate_initial_population(
    sample_size=100,
    pattern_size=3,
    feat_idx_pool=fss,
    lag_allowance=lag_allowance,
    skew_lag_prob=False
)

print(len(population[0]._patterns))

print('beginning generational loop')

#begin generational loop
for generation in range(generations):

	print(f'checking population fitness: GEN # {generation+1}')

	returns, kelsch_ratio = _2.fitness(
		arr_close=arr_close,
		arr_low=arr_low,
		data = dataset,
		genes= population,
		hold_for=hold_for,
		lag_allow=lag_allowance,
		specific_data='form_519',
		log_normalize=True
	)

	print('associating population fitness',end='')

	unsorted_population = _2.associate(
		genes=population,
		returns=returns,
		kelsch_ratio=kelsch_ratio
	)

	print('sorting population')

	criteria = 'kelsch_ratio'

	population = _2.sort_population(
		population=unsorted_population,
		criteria=criteria
	)

	avg, top = _2.simple_generational_stat_output(population,criteria)
	print(f"Generation {generation+1} ({criteria}): AVG {avg}, BEST {top}, FROM {len(population)} GENES.")

	print('collecting elites..',end='')

	elites = _3.collect_elite(
		sorted_population=population,
		filter_criteria=1
	)

	print('collecting parents..',end='')


	parents = _4.collect_parents(
		sorted_population=population,
		criteria=criteria,
		num_parents=25
	)

	print('reproducing..',end='')

	family = _5.reproduce(
		parents=parents,
		mode='linear'
	)

	print('shuffling population..',end='')

	shuffled_family = _1.shuffle_population(
		population=family
	)

	print('mutating family',end='')

	#print(f'parents: LEN={len(parents)}')
	#print(f'family: LEN={len(family)}')


	mutated_family = _6.mutation_round(
		shuffled_population=shuffled_family,
		partial_mutation_prob=0.000,
		pattern_mutation_prob=0.000,
		feat_idx_pool=fss	
	)

	print('combining populations')

	population = _1.combine_populations(
		populations=[elites, mutated_family]
	)

In [ ]:
import _0_gene as _0
import _9_utility as  util
reload(_0)
reload(util)
reload(_2)

_2.show_best_gene_patterns(elites,criteria,fssd)